# We recreate the cases where Extended Kalman Filter diverges, and explain why it diverge.

In [50]:

%matplotlib inline
import numpy as np
from scipy import special
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

import pickle as pkl
import networkx as nx
from matplotlib import style
import jax.numpy as jnp
from jax import jacfwd, jit,grad
from functools import partial


from Robot import Robot
from virtual_sensor import virtual_sensor
from utils.DynamicFilters import getDynamicFilter,joint_meas_func
from utils.FIMPathPlanning import FIM_ascent_path_planning 
from utils.dLdp import FIM,FIM_mix,analytic_dLdp,analytic_dhdz
from utils.regions import CircleExterior
from tracking_log import logger



%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
class DistributedEKF:
    """
        The distributed version of Extended Kalman Filter, used by a single robot to estimate the source location from the information shared by its local teammates via comm.
        
        The major difference of this estimator from the conventional EKF is that the measurement function may be changing in time, so are the dimensions of y_t and p_t, due to 
        the change in the group of local teammates, as a result of changing comm. network topology.
        
        The estimator assumes a constant-velocity source movement model, with the velocity to be the fundamental uncertainty.
    """
    def __init__(self,q_0,R_mag=1,Q_mag=1):
        '''
        q_0 should be a vector, the initial guess of source position.
        
        R_mag,Q_mag should be scalars.
        '''
        
        self.q = q_0 # Mean
        self.qdot = np.zeros(self.q.shape) # Velocity Mean
        
        self.z = np.hstack([self.q,self.qdot]) # The source state
        
        
        self.qdim = len(self.q)
        
        self.P = np.eye(len(self.z)) # Covariance of [q,qdot]. Initialized to be the identity matrix
        
        self.R_mag = R_mag 
        self.Q_mag = Q_mag
        self._K = 0
        self._C=0
        self._yhat = 0
    
    def dfdz(self,z):
        n = len(z)//2
        O =np.zeros((n,n))
        I=np.eye(n)
        return np.vstack([np.hstack([I,I]),np.hstack([O,I])])

    def f(self,z):
        """
            The constant velocity model.
        """
        A = self.dfdz(z)
        
        return A.dot(z)
    def update(self,h,dhdz,y,p):
        """
        h is a function handle h(z,p), the measurement function that maps z,p to y.
        dhdz(z,p) is its derivative function handle.
        
        y is the actual measurement value, subject to measurement noise.
        
        p is the positions of the robots.
        """
        A = self.dfdz(self.z)
        C = dhdz(self.z,p)
        R = self.R_mag*np.eye(len(y))
        Q = self.Q_mag*np.eye(len(self.z))
        
        # The Kalman Gain
        K = A.dot(self.P).dot(C.T).dot(    np.linalg.inv(C.dot(self.P).dot(C.T)+R)      )
        self._K=K
        self._C=C

        # Mean and covariance update
        self._yhat = h(self.z,p)
        self.z = self.f(self.z)+K.dot(y-h(self.z,p))
        
        print("K..",K.dot(C.dot(self.P).dot(C.T)+R).dot(K.T))
        print('APA^T',A.dot(self.P).dot(A.T))
        print('Q',Q)
        print('P prior',self.P)
        self.P = A.dot(self.P).dot(A.T)+ Q- K.dot(C.dot(self.P).dot(C.T)+R).dot(K.T)
        print('P post',self.P)
        print('-------------------')
    def update_and_estimate_loc(self,h,dhdz,y,p):
        if not np.any(y == np.inf):
            self.update(h,dhdz,y,p)

        return self.z[:len(self.q)]

    
        
        

In [6]:
def fc_comm_network(i,qs,ps):
    """
        Given the iteration number i, source location qs, sensor location ps,
        return the current communication network G.
        
        This variation returns a fully connected network with number of nodes=len(ps),
        without self-loops.
    """
    A = np.ones([len(ps),len(ps)])-np.eye(len(ps))
    return nx.from_numpy_matrix(A,create_using=nx.DiGraph)

In [7]:
def circulant(i,q,p,prev,post,undirected=False):
    """
        Generate a circulant graph with len(p) nodes, node i connected with [i-prev:i+post],i-prev and i+post included but self-loop eliminated.
    """
    n = len(p)
    G = nx.DiGraph()
    edges = [(j%n,i) for i in range(n) for j in range(i-prev,i+post+1)]
    G.add_edges_from(edges)
    G.remove_edges_from(nx.selfloop_edges(G))
    if undirected:
        G = G.to_undirected()
    return G
    

In [8]:
def fixed_size_network(i,qs,ps,group_sizes):
    """
        Given the iteration number i, source location qs, sensor location ps,
        return the current communication network G.
        
        Return a disconnected graph with the specified list of group sizes. 
    """
    n=len(ps)
    assert(n==np.sum(group_sizes))
    components = [nx.complete_graph(s) for s in group_sizes]
    network = nx.disjoint_union_all(components)
    return nx.to_directed(network)

In [67]:
def main(initial_locs,comm_network_generator,N_iter):
    """
    Note: 
    comm_network_generator(i,qs,ps) is the expected signature of the second argument.
    
        Given the iteration number i, source location qs, sensor location ps,
        comm_network_generator return the current communication network G.
        
    """
    # Set up mobile sensors and sources
    sensor_names = ["mobile_sensor_{}".format(i+1) for i in range(len(initial_locs))]
    mobile_sensors = [Robot(loc,name) for loc,name in zip(initial_locs,sensor_names)]

    src_locs = [[8.,8]]
    src_names = ["source_{}".format(i+1) for i in range(len(src_locs))]
    source = [Robot(loc,name) for loc,name in zip(src_locs,src_names)]

    # Set up virtual sensors
    C1=0
    C0=0
    k=1
    b=-2
    std = 0.1
    vs = virtual_sensor(C1,C0,b,k,std)

    # Set up location estimator
    n_sen = len(mobile_sensors)
    n_src = len(source)

    C1s=C1*np.ones(n_sen)
    C0s = C0*np.ones(n_sen)
    ks = k * np.ones(n_sen)
    bs = b*np.ones(n_sen)
    
#     q_0 = np.random.rand(len(mobile_sensors),2)*10
    
    
    q_0 = np.array([[6.,6.0] for _ in range(len(sensor_names))])

    estimators = [DistributedEKF(q,R_mag=0.1,Q_mag=0.1)  for q in q_0]

    # Set up data logger
    log = logger(sensor_names,src_names)
    log.est_locs_log =dict({"Est {}".format(sensor):[] for sensor in sensor_names})
    
    # Assume fixed topology, pre-set the measurement model and waypoint planner(expensive to be created in the main loop)
    ps = np.array([r.loc for r in mobile_sensors])
    qs = np.array([s.loc for s in source])
    G = comm_network_generator(0,qs,ps)
   
    hs = []
    dhdzs = []
    f_dLdps = []
    Ks = {m.name:[] for m in mobile_sensors}
    Cs = {m.name:[] for m in mobile_sensors}
    Ps = {m.name:[] for m in mobile_sensors}
    ys = {m.name:[] for m in mobile_sensors}
    yhats = {m.name:[] for m in mobile_sensors}
    for i in G.nodes():
        N_i = [i]+list(G[i])     
       
        C1s_i=C1s[N_i]
        C0s_i = C0s[N_i]
        ks_i = ks[N_i]
        bs_i = bs[N_i]
        h=partial(joint_meas_func,C1s_i,C0s_i,ks_i,bs_i)# Freeze the coefficients, the signature becomes h(z,ps)
        dhdzs.append(partial(analytic_dhdz,C1s_i,C0s_i,ks_i,bs_i))
        hs.append(h)
        
        f_dLdps.append(partial(analytic_dLdp,C1s=C1s_i,C0s=C0s_i,ks=ks_i,bs=bs_i))
#         f_dLdps.append(dLdp(C1s=C1s_i,C0s=C0s_i,ks=ks_i,bs=bs_i))


    
    # Enter main loop

    for _ in range(N_iter):
        # Measure
        ps = np.array([r.loc for r in mobile_sensors])
        qs = np.array([s.loc for s in source])
          
        y = vs.measurement(qs,ps)

        # Set up the current communication network.
        G = comm_network_generator(_,qs,ps)
        log.comm_network.append(G)
  
        # Record
        for s in source:
            log.src_locs[s.name].append(s.loc)



        # Estimate, plan and move a step for each robot.
        for i in G.nodes():
            # Get all robots queriable in the comm. network, including i.
            # Note: i's place within N_i is always at index 0
            
            """ The following line gets all the immediate neighbors rather than all ancestors, which is way this Notebook is prefixed with P2P(point to point)"""
            if not i==0:
                continue
            N_i = [i]+list(G[i]) 
            print(N_i)
            n_sen = len(N_i)

            # Simulate query of information from other robots. 
            ps_i =ps[N_i]
            y_i = y[N_i]
            print('ps_i',ps_i)
            print('y_i',y_i)

            # Estimate
            zhat = estimators[i].z
            qhat=estimators[i].update_and_estimate_loc(hs[i],dhdzs[i],y_i,ps_i)

#             # Set up waypoint planner
#             f_dLdp=f_dLdps[i]
            
#             planning_timesteps = 1
#             max_linear_speed = 0.22
#             planning_dt = 1
#             epsilon = 0.5
#             hard_boundary = CircleExterior(qs[0],0.5)
#             # The FIM waypoint planning  
#             waypoints=FIM_ascent_path_planning(f_dLdp,qhat,ps_i,n_sen,\
#                             planning_timesteps,\
#                             max_linear_speed,\
#                             planning_dt,\
#                             epsilon,hard_boundary)

            # Note: i's place within N_i is always at index 0
            # Move to the 0th waypoint
#             mobile_sensors[i].update_loc(waypoints[0][0])
            m = mobile_sensors[i]

            log.sensor_locs[m.name].append(m.loc)
            log.est_locs_log["Est {}".format(m.name)].append(zhat)
            Ks[m.name].append(estimators[i]._K)
            Cs[m.name].append(estimators[i]._C)
            Ps[m.name].append(estimators[i].P)
            ys[m.name].append(y_i)
            yhats[m.name].append(estimators[i]._yhat)
    out=log.export()
    out['Ks']=Ks
    out['Cs']=Cs
    out['Ps']=Ps
    out['ys']=ys
    
    out['yhats']=yhats
    return out




In [68]:
# src_locs = np.array([[8.,8.]])
# sensor_center = src_locs
# r=0.5
# thetas = np.pi * 2 * np.linspace(0.5,0.75,3)
# initial_locs =  r*np.vstack([np.sin(thetas),np.cos(thetas)]).T+sensor_center


r=0.5
thetas = np.pi * 2 * np.linspace(0,1,3,endpoint=False)
initial_locs = r*np.vstack([np.sin(thetas),np.cos(thetas)]).T+np.array([[8.,8]])
# comm_network=lambda i,q,p:random_switch_netowrk(i,q,p,conn_rate = 0.5)

# comm_network=lambda i,q,p:fc_comm_network(i,q,p)

# comm_network=lambda i,q,p: distance_based_network(i,q,p,C=0.3,n=2.1)

comm_network=lambda i,q,p:circulant(i,q,p,prev=1,post=0,undirected=True)  
log = main(initial_locs,comm_network,200)

# Plotting
record = log

filepath = "EKF-Debug.pkl"
with open(filepath,'wb') as file:
    pkl.dump(record,file)

[0, 2, 1]
ps_i [[8.        8.5      ]
 [7.5669873 7.75     ]
 [8.4330127 7.75     ]]
y_i [4.06026412 4.00869288 4.08295723]
K.. [[0.11839122 0.12214728 0.         0.        ]
 [0.12214728 0.1310022  0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
APA^T [[2. 0. 1. 0.]
 [0. 2. 0. 1.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]]
Q [[0.1 0.  0.  0. ]
 [0.  0.1 0.  0. ]
 [0.  0.  0.1 0. ]
 [0.  0.  0.  0.1]]
P prior [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
P post [[ 1.98160878 -0.12214728  1.          0.        ]
 [-0.12214728  1.9689978   0.          1.        ]
 [ 1.          0.          1.1         0.        ]
 [ 0.          1.          0.          1.1       ]]
-------------------
[0, 2, 1]
ps_i [[8.        8.5      ]
 [7.5669873 7.75     ]
 [8.4330127 7.75     ]]
y_i [3.92846307 3.97102999 4.03496341]
K.. [[0.01642117 0.01630687 0.00574217 0.00572862]
 [0.01630687 0.01656967 0.0057074  0.0058157 ]
 [0.00574217 0

## The following discussions are made using complete communication network

The experiment shows given enough number of iterations, almost always some of the filters diverge. After plotting out the velocity estimation, we found that the divergence corresponds to the ones with non-zero steady-state velocity estimations. Some version of consensus filters can eliminate the divergence in velocity estimation through a "majority vote" on zero velocity.

The next question is, why the filter update does not correct the non-zero steady-state velocity? It looks like the velocity remains constant non-zero, which means the feedback on the erroneous velocity is almost nothing. Is it corresponding to the Kalman Gain?

**Looking at the figures of Ks values, indeed the feedback Ks converges to 0 at steady state, even for the divergent sensors.**

**Looking at the figures of Cs values, indeed the Cs for the divergent sensors are always very small. Which is reasonable if their zhat is very far away from the current ps.**

Looking at the figures of Ps values, the Ps for the divergent sensors blows up.

Looking at the figures of CPC^T values, the smallness of C values dominates over the largeness of P values for the divergent sensors, so CPC^T is very small, so $(CPC^T+R)^{-1}\approx R^{-1}=I$, as we set $R=I$. For convergent sensors, $CPC^T$ is significantly greater than zero.

Looking at the figures of CPPC^T(the square of PC^T) values, it is very small in steady state for both divergent and convergent sensors. So the dominating factor to make Ks small for divergent sensors is the CPC^T term going to near zero, making the (CPC^T+R)^-1 term close to I while PC^T term close to zero, and therefore making K close to zero, therefore unable to provide any feedback in the steady state.

An intuition is if we set $R$ to be much smaller, so that its magnitude is close to the noise magnitude we are using ($std = 0.1$), then the $(CPC^T+R)^{-1}$ term will be larger when $CPC^T$ is small, therefore can help construct a non-trivial $K$ at the steady state. 

**And indeed, using a smaller $R$ gets rid of the divergent behavior after a few trial. But the convergence rate is poor this time. So it seems the fundamental issue in our EKF is indeed a parameter tuning issue, rather than a linearization issue.**

## When change to sparse network

When change to sparse network as those we used in other experiments, convergence is harder, and there is in general no steady state. $PC^T$ and $CPC^T$ value remains small for most of the times, and reducing the $R$ magnitude has less a significant effect. 

**But still, the divergent sensor has the trademark divergent $P$ matrix, and still its $K$ is vanishing, and estimated velocity stays non-zero.**

## Reproduce the divergent behavior in a static setting

I think the main reason that drives the estimation update divergent, especially those drives the C values to vanish and consequently result in a vanishing K, **is we linearize H at a zhat that is very far away from the true value.** That is equivalent as placing the sensors close to the source, but give then initial guesses that are far-far away from the true source. EKF does not guarantee global convergence, that may be the core of the problem.

I will verify this conjecture by doing a static estimation simulation, with a small sensor source distance but far-away initial guess.

The conjecture is roughly verified using a equi-distance formation around the source. When the initial guess is within the envolope formed by the sensors, the static estimation coverges well. When the initial guess is outside of the envolope formed by the sensors, the estimation diverges. Note that when the sensors move to converge to the source, their envolope become smaller, so it is more likely their current guess is outside of their envolope, resulting in an eventual divergence.

**The next thing to do is to construct more such cases, and display them in a well formatted notebook, so that the conjecture is fully supported by data: the quality of zhat in this iteration determines the quality of linearization in the next iteration, therefore determines the quality of zhat in the next iteration. If the error exceeds a certain threshold, the linearization will break**

**The obvious solutions are 1) Linearize each row of C using your neighbors' zhat, rather than your own. 2) Avoid linearization using zhat, but write C as a function of y values(if possible). 3) Correct the estimation using hard constraints/non-dynamical estimation.**